In [30]:
from pymongo import MongoClient
import pprint

import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time

In [31]:
roster_urls = {}
roster_urls['unc'] = 'https://goheels.com/sports/womens-basketball/roster/'
roster_urls['duke'] = 'https://goduke.com/sports/womens-basketball/roster/'
roster_urls['miami'] = 'https://hurricanesports.com/sports/womens-basketball/roster/'
roster_urls['nc_state'] = 'https://gopack.com/sports/womens-basketball/roster/'

#List of desired seasons of data - will combine with roster_urls to scrape
seasons = ['2009-10', '2010-11', '2011-12', '2017-18', '2018-19', '2019-20']

In [33]:
#Grab requests and input into MongoDB - commented out after completing once
client = MongoClient('localhost', 27017)
db = client.wbb2
rosters = db.rosters

for season in seasons:
    for team, url in roster_urls.items():
        r = requests.get(url + season)
        rosters.insert_one({'team': team,
                            'season': season,
                            'html': r.text})
        time.sleep(2)

At this point I have a cursor that contains all the HTML files containing each team's roster for every season.

Now I need to separate out the html but make sure to tie it to the correct team & season.

Fields I want are:
-heights
-names
-positions
-class_yr

Update MongoDB fields to have the keys with the fields you want with values that consist of the parsed out values.

*****Note*****
-Ran into an issue with parsing the data from Mongo.  I saved the html files in mongo using r.content.  R.content saves the files as bytes rather than strings (r.text).  Didn't find a quick solution to deal with bytes so pulled the html again into the 'wbb2' database and saved it using r.text.

--Update--
The data is fine for unc for the 2019-2020, 2018-19, 2017-2018 seasons, but I suspect the HTML changed so now I have to figure that out.  This wasn't apparent the first time because I was checking the oldest seasons as opposed to the last seasons.

-Iterate from the end of the cursor and once the lists fail to populate, use a different scheme for parsing.

In [57]:
cursor = rosters.find({})

In [58]:
all_teams = []
for item in cursor:
    if item['team'] == 'unc':
        soup = BeautifulSoup(item['html'], 'html.parser')
        #run unc code
        heights = [item.text for item in soup.find_all(class_='height')]
        names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
        positions = [item.text for item in soup.find_all(class_='rp_position_short')]
        class_yr = [item.text for item in soup.find_all(class_='roster_class')]
        all_teams.append({'team': item['team'],
                          'season': item['season'],
                          'heights': heights,
                          'names': names,
                          'positions': positions,
                          'class_yr': class_yr})
    elif item['team'] == 'duke':
        soup = BeautifulSoup(item['html'], 'html.parser')
        #run duke code
        heights = [item.text for item in soup.find_all(class_='height')]
        names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
        positions = [item.text for item in soup.find_all(class_='rp_position_short')]
        class_yr = [item.text for item in soup.find_all(class_='roster_class')]
        all_teams.append({'team': item['team'],
                          'season': item['season'],
                          'heights': heights,
                          'names': names,
                          'positions': positions,
                          'class_yr': class_yr})
    elif item['team'] == 'miami':
        soup = BeautifulSoup(item['html'], 'html.parser')
        #run miami code
        heights = [item.text for item in soup.find_all(class_='height')]
        names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
        positions = [item.text for item in soup.find_all(class_='rp_position_short')]
        class_yr = [item.text for item in soup.find_all(class_='roster_class')]
        all_teams.append({'team': item['team'],
                          'season': item['season'],
                          'heights': heights,
                          'names': names,
                          'positions': positions,
                          'class_yr': class_yr})
    else:
        soup = BeautifulSoup(item['html'], 'html.parser')
        #run nc_state code
        heights = [item.text for item in soup.find_all(class_='height')]
        names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
        positions = [item.text for item in soup.find_all(class_='rp_position_short')]
        class_yr = [item.text for item in soup.find_all(class_='roster_class')]
        all_teams.append({'team': item['team'],
                          'season': item['season'],
                          'heights': heights,
                          'names': names,
                          'positions': positions,
                          'class_yr': class_yr})

In [62]:
blank_teams = []
for item in all_teams:
    if item['heights'] == []:
        blank_teams.append({'team': item['team'],
                            'season': item['season']
        })
blank_teams

[{'team': 'unc', 'season': '2009-10'},
 {'team': 'miami', 'season': '2009-10'},
 {'team': 'nc_state', 'season': '2009-10'},
 {'team': 'unc', 'season': '2010-11'},
 {'team': 'miami', 'season': '2010-11'},
 {'team': 'nc_state', 'season': '2010-11'},
 {'team': 'unc', 'season': '2011-12'},
 {'team': 'miami', 'season': '2011-12'},
 {'team': 'nc_state', 'season': '2011-12'}]

For some weird reason, the HTML was the same with the final 3 years

In [22]:
'''
unc parsing code:

****NEW CODE****
#Try new code instead

heights = [item.text for item in soup.find_all(class_='height')]
names = [item.text for item in soup.find_all(class_='sidearm-table-player-name')]
positions = [item.text for item in soup.find_all(class_='rp_position_short')]
class_yr = [item.text for item in soup.find_all(class_='roster_class')]
season = item['season']
team = item['team']

****OLD CODE****

heights = soup.find_all(class_='height')
names = soup.find_all(class_='sidearm-table-player-name')
positions = soup.find_all(class_='rp_position_short')
class_yr = soup.find_all(class_='roster_class')

new_heights = [item.text for item in heights]
new_names = [item.text for item in names]
new_positions = [item.text for item in positions]
new_class_yr = [item.text for item in class_yr]
'''


Okay, figured out how to scrape the North Carolina 2019-2020 page.

Now I need to figure out how to move from year to year to scrape the roster pages for the other years.

-Looks like each roster webpage can be navigated by adding the season to the end of the URL (e.g. if I wanted the 2015-2016 roster, then 'https://goheels.com/sports/womens-basketball/roster') would become 'https://goheels.com/sports/womens-basketball/roster/2015-16'.

In [56]:
#Create dictionary to store key/value pairs for teams & roster webpage
teams_rosters = {}
teams_rosters['unc'] = 'https://goheels.com/sports/womens-basketball/roster/'
teams_rosters['duke'] = 'https://goduke.com/sports/womens-basketball/roster/'
teams_rosters['miami'] = 'https://hurricanesports.com/sports/womens-basketball/roster/'
teams_rosters['nc_state'] = 'https://gopack.com/sports/womens-basketball/roster/'

#List of desired seasons of data - will combine with teams_rosters to scrape
seasons = ['2009-10', '2010-11', '2011-12', '2017-18', '2018-19', '2019-20']

In [58]:
teams_rosters['unc'] + seasons[0]

'https://goheels.com/sports/womens-basketball/roster/2009-10'

In [62]:
all_requests = []
for k, v in teams_rosters.items():
    for season in seasons:
        url = v + season
        r = requests.get(url)
        all_requests.append(r)
        
        


200
200
200
200


In [66]:
soup = BeautifulSoup(all_requests[1].text, 'html.parser')
print(soup.prettify())

<!DOCTYPE doctype html>
<html class="wide roster-page sport wbball" id="ctl00_html" lang="en">
 <head>
  <script>
   window.client_hostname = "goduke.com";window.server_name = "1042434-LB24";window.client_division = "DI";window.targetting_criteria = {"page_template":"roster-list","sport_name":"wbball","sport_name_custom":"wbball","site":"duke","cbs_site_code":"duke","content_id":null,"sect":"wbaskbl"};window.client_shortname = "duke";window.dfp_network_code = "21708449227";window.img_dfp_unit_name = "/cstv.duke";window.block_dfp = false;window.reactions = null;window.cbs_ads_activated = false;window.site_colors = {"safe_text_white":"#454444","safe_text_black":"#ffffff","primary_background":"#013088","primary_text":"#ffffff","secondary_background":"#ffffff","secondary_text":"#013088"};
                    window.utagData = function() {
                        return {
                            embedPageUrl: location.href,
                            brandPlatformId: "sidearm_site_" + 